In [29]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
warnings.filterwarnings('ignore')

# Load the dataset
company = pd.read_csv("companies.csv")
print(company.head())
print(company.describe())
print(company.columns)


        id  Unnamed: 0.1 entity_type  entity_id  parent_id  \
0      c:1             0     Company          1        NaN   
1     c:10             1     Company         10        NaN   
2    c:100             2     Company        100        NaN   
3  c:10000             3     Company      10000        NaN   
4  c:10001             4     Company      10001        NaN   

                 name     normalized_name                    permalink  \
0            Wetpaint            wetpaint            /company/wetpaint   
1             Flektor             flektor             /company/flektor   
2               There               there               /company/there   
3             MYWEBBO             mywebbo             /company/mywebbo   
4  THE Movie Streamer  the movie streamer  /company/the-movie-streamer   

     category_code     status  ... first_milestone_at last_milestone_at  \
0              web  operating  ...         2010-09-05        2013-09-18   
1      games_video   acquired  .

In [30]:
company.columns

Index(['id', 'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 'name',
       'normalized_name', 'permalink', 'category_code', 'status', 'founded_at',
       'closed_at', 'domain', 'homepage_url', 'twitter_username', 'logo_url',
       'logo_width', 'logo_height', 'short_description', 'description',
       'overview', 'tag_list', 'country_code', 'state_code', 'city', 'region',
       'first_investment_at', 'last_investment_at', 'investment_rounds',
       'invested_companies', 'first_funding_at', 'last_funding_at',
       'funding_rounds', 'funding_total_usd', 'first_milestone_at',
       'last_milestone_at', 'milestones', 'relationships', 'created_by',
       'created_at', 'updated_at', 'lat', 'lng', 'ROI'],
      dtype='object')

In [31]:
# A. Data Cleaning
# 1. Delete irrelevant & redundant information.

# 1.a. Delete 'region', 'city', 'state_code'
company = company.drop(['region', 'city', 'state_code'], axis=1)

# 1.b. Delete redundant columns
redundant_cols = ['id', 'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 
                  'created_by', 'created_at', 'updated_at']
company = company.drop(redundant_cols, axis=1)

# 1.c. Delete irrelevant features
irrelevant_cols = ['domain', 'homepage_url', 'twitter_username', 'logo_url', 
                   'logo_width', 'logo_height', 'short_description', 
                   'description', 'overview','tag_list', 'name', 
                   'normalized_name', 'permalink', 'invested_companies']
company = company.drop(irrelevant_cols, axis=1)

# 1.d. Delete duplicate values
company = company.drop_duplicates()

# 1.e. Delete those which have more than 98% null values
threshold = 0.02 * company.shape[0]
company = company.dropna(thresh=threshold, axis=1)


In [32]:
# 2. Remove noise or unreliable data (missing values and outliers)

# 2.a. Delete instances with missing values for 'status', 'country_code', 'category_code', and 'founded_at'
company = company.dropna(subset=['status', 'country_code', 'category_code', 'founded_at'])

# 2.b. Delete outliers for 'funding_total_usd' and 'funding_rounds'
# For funding_total_usd
Q1_funding_total = company['funding_total_usd'].quantile(0.25)
Q3_funding_total = company['funding_total_usd'].quantile(0.75)
IQR_funding_total = Q3_funding_total - Q1_funding_total
lower_bound_funding_total = Q1_funding_total - 1.5 * IQR_funding_total
upper_bound_funding_total = Q3_funding_total + 1.5 * IQR_funding_total



In [33]:
# For funding_rounds
Q1_rounds = company['funding_rounds'].quantile(0.25)
Q3_rounds = company['funding_rounds'].quantile(0.75)
IQR_rounds = Q3_rounds - Q1_rounds
lower_bound_rounds = Q1_rounds - 1.5 * IQR_rounds
upper_bound_rounds = Q3_rounds + 1.5 * IQR_rounds

# Remove outliers
company = company[(company['funding_total_usd'] >= lower_bound_funding_total) & 
                  (company['funding_total_usd'] <= upper_bound_funding_total)]
company = company[(company['funding_rounds'] >= lower_bound_rounds) & 
                  (company['funding_rounds'] <= upper_bound_rounds)]



In [34]:
# B. Data Transformation
# 1. Changes in original data

# 1.a. Convert dates to years
company['founded_at'] = pd.to_datetime(company['founded_at']).dt.year
company['closed_at'] = pd.to_datetime(company['closed_at']).dt.year
company['first_funding_at'] = pd.to_datetime(company['first_funding_at']).dt.year
company['last_funding_at'] = pd.to_datetime(company['last_funding_at']).dt.year
company['first_milestone_at'] = pd.to_datetime(company['first_milestone_at']).dt.year
company['last_milestone_at'] = pd.to_datetime(company['last_milestone_at']).dt.year

# 1.b. Generalize the categorical data
# Generalize 'category_code'
category_count = company['category_code'].value_counts()
top_categories = category_count[:10].index
company['category_code'] = np.where(company['category_code'].isin(top_categories), 
                                    company['category_code'], 'other')

In [35]:
# One-hot encode 'category_code'
category_dummies = pd.get_dummies(company['category_code'], prefix='category')
company = pd.concat([company, category_dummies], axis=1)
company = company.drop('category_code', axis=1)

# Generalize 'country_code'
country_count = company['country_code'].value_counts()
top_countries = country_count[:10].index
company['country_code'] = np.where(company['country_code'].isin(top_countries), 
                                   company['country_code'], 'other')

# One-hot encode 'country_code'
country_dummies = pd.get_dummies(company['country_code'], prefix='country')
company = pd.concat([company, country_dummies], axis=1)
company = company.drop('country_code', axis=1)


In [36]:
# 2. Create new variables

# 2.a. Create new feature isClosed
company['isClosed'] = np.where(company['status'].isin(['operating', 'ipo']), 1, 0)

# 2.b. Create active_days
company['closed_at'] = np.where(company['closed_at'].isna() & 
                                company['status'].isin(['operating', 'ipo']), 2021, 
                                company['closed_at'])
company['closed_at'] = np.where(company['closed_at'].isna(), 0, company['closed_at'])
company['active_days'] = company['closed_at'] - company['founded_at']


In [37]:
# Remove the 'closed_at' column
company = company.drop('closed_at', axis=1)

In [38]:
# Final null check and filling missing values with mean for numerical data
numeric_cols = company.select_dtypes(include=['number']).columns
company[numeric_cols] = company[numeric_cols].fillna(company[numeric_cols].mean())


In [39]:
# Save cleaned data
company.to_csv('cleaned_companies.csv', index=False)

# Display the cleaned dataset
print(company.head())

       status  founded_at first_investment_at last_investment_at  \
13   acquired        2007                 NaN                NaN   
20  operating        2003                 NaN                NaN   
26  operating        2003                 NaN                NaN   
32  operating        2010                 NaN                NaN   
36  operating        2006                 NaN                NaN   

    investment_rounds  first_funding_at  last_funding_at  funding_rounds  \
13               10.0            2008.0           2008.0             1.0   
20               10.0            2011.0           2012.0             3.0   
26               10.0            2003.0           2003.0             1.0   
32               10.0            2011.0           2011.0             1.0   
36               10.0            2011.0           2012.0             2.0   

    funding_total_usd  first_milestone_at  ...  country_ESP  country_FRA  \
13          5000000.0         2008.000000  ...        Fals